In [17]:
import pandas as pd
import pyodbc
from SchedulePy.Path import make_directory, current_path_and_path_list
from pathlib import Path


In [18]:
current_path, path_list = current_path_and_path_list()
datapath = make_directory(str(current_path)+'\\Data\\')

In [19]:
class Uploader:
    def __init__(self, abspath, conn):
        self.abspath = abspath
        self.conn = conn
        
    def create_table(self,db=None):
        self.db = db
        if not self.db:
            raise ValueError('db parameter is required')
        self.db_name = self.db.split('.')[0]
        self.schema = self.db.split('.')[1]
        self.table_name = self.db.split('.')[2]

        cursor = self.conn.cursor()
        cursor.execute(f"USE [{self.db_name}]")
        print(f"Switched to database: {self.db_name}")

        # Check if that table already exists
        cursor.execute(f"SELECT COUNT(*) FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = '{ self.table_name}' AND TABLE_SCHEMA = '{self.schema}'")
        table_exists = cursor.fetchone()[0]
        if not table_exists:
            df = self.__load_file()
            # generate SQL column definitions based on DataFrame's dtypes
            column_definitions = self.__generate_sql_columns(df)
            
            print(f"Columns{column_definitions}")
            return df
        
        # hidden class method to load the file based on its extension
    def __load_file(self):
        # mapping of file extensions to pandas loader functions
        loaders = {
            '.csv': pd.read_csv,
            '.xlsx': pd.read_excel,
            '.pkl': pd.read_pickle,
            '.feather': pd.read_feather
            # add more extensions and loaders as needed
        }

        # extracting the file extension using pathlib
        file_extension = Path(self.abspath).suffix.lower()

        # get the appropriate loader function or raise an error if unsupported
        loader = loaders.get(file_extension)
        if loader:
            return loader(self.abspath)

        raise ValueError(f"Unsupported file format: {file_extension}")


    # hidden class method to generate SQL column definitions from DataFrame
    def __generate_sql_columns(self, df):
        # mapping pandas dtypes to SQL types
        dtype_mapping = {
            'int64': 'INT',
            'float64': 'FLOAT',
            'object': 'NVARCHAR(MAX)',
            'datetime64[ns]': 'DATETIME',
            'bool': 'BIT',
            # add more mappings as necessary
        }

        # creating the column definitions string
        column_definitions = ', '.join([
            f"{col} {dtype_mapping.get(str(dtype), 'NVARCHAR(MAX)')}"
            for col, dtype in df.dtypes.items()
        ])

        return column_definitions
        
        
    

In [20]:
# Connect to your SQL Server database
# conn = pyodbc.connect('Driver={SQL Server};'
#                       'Server=10.102.103.104;'
#                       'Database=something;')

conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=10.195.103.194;'
                      'Database=Master;'
                      'Trusted_Connection=yes;')

uploader = Uploader(abspath=r'C:\Users\kristinedzneladze\Desktop\SchedulePy\Data\pnk.feather',conn=conn)
uploader.create_table("WH_ScoringReports.idk.pinkfloyd")

Switched to database: WH_ScoringReports
ColumnsAlbum NVARCHAR(MAX), Release Date DATETIME, Sales (Millions) FLOAT, Genre NVARCHAR(MAX), Duration (Minutes) FLOAT, Label NVARCHAR(MAX), Grammy Nominations FLOAT, Critics Score FLOAT, Recording Start DATETIME


,Album,Release Date,Sales (Millions),Genre,Duration (Minutes),Label,Grammy Nominations,Critics Score,Recording Start
0,The Dark Side of the Moon,1973-03-01,45.0,Progressive Rock,43.0,Harvest,1.0,9.5,1972-06-01
1,Wish You Were Here,1975-09-12,20.0,Progressive Rock,44.0,Harvest,NaN,9.2,NaT
2,The Wall,1979-11-30,30.0,Rock Opera,81.0,Columbia,6.0,NaN,1978-01-01
3,None,NaT,NaN,Rock,NaN,Harvest,1.0,8.7,1976-04-01
4,Animals,1977-01-23,12.0,None,41.0,None,NaN,8.9,1976-04-01


In [9]:
# conn = pyodbc.connect('Driver={SQL Server};'
#                       'Server=10.195.103.194;'
#                       'Database=WH_ScoringReports;'
#                       'Trusted_Connection=yes;')


In [10]:
import pandas as pd
import numpy as np

# Generating sample data for a DataFrame about Pink Floyd
data = {
    'Album': ['The Dark Side of the Moon', 'Wish You Were Here', 'The Wall', np.nan, 'Animals'],
    'Release Date': ['1973-03-01', '1975-09-12', '1979-11-30', np.nan, '1977-01-23'],
    'Sales (Millions)': [45.0, 20.0, 30.0, np.nan, 12.0],
    'Genre': ['Progressive Rock', 'Progressive Rock', 'Rock Opera', 'Rock', np.nan],
    'Duration (Minutes)': [43.0, 44.0, 81.0, np.nan, 41.0],
    'Label': ['Harvest', 'Harvest', 'Columbia', 'Harvest', np.nan],
    'Grammy Nominations': [1, np.nan, 6, 1, np.nan],
    'Critics Score': [9.5, 9.2, np.nan, 8.7, 8.9],
    'Recording Start': ['1972-06-01', np.nan, '1978-01-01', '1976-04-01', '1976-04-01']
}

# Converting date columns to datetime
data['Release Date'] = pd.to_datetime(data['Release Date'], errors='coerce')
data['Recording Start'] = pd.to_datetime(data['Recording Start'], errors='coerce')

# Creating the DataFrame
pink_floyd_df = pd.DataFrame(data)
pink_floyd_df.to_pickle(datapath+'pnk.pkl')
pink_floyd_df.to_excel(datapath+'pnk.xlsx')
pink_floyd_df.to_csv(datapath+'pnk.csv')
pink_floyd_df.to_feather(datapath+'pnk.feather')
